###### installing some packages

In [ ]:
! pip install -U scikit-learn scipy matplotlib

###### imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from matplotlib import transforms
import scipy
import scipy.stats as st
from scipy.integrate import odeint
import networkx as nx
import random
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from SequenceGenerator import MultiSequenceGenerator
from PoissonParetoSimulator import PoissonParetoBurstProcessSimulator
import matplotlib.mlab as mlab
from sklearn.neighbors import KernelDensity
import copy

# Rewiring model graph

## implementation

In [ ]:
class WiringModelGraph():
    def draw_graph_for_time_slot(self):
        fig = plt.figure(figsize=(50, 50))
        degrees = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        self.degree_per_slot.append(degrees)
        node_size = degrees*100
        
        pos=nx.spring_layout(self.graph, k=0.15, iterations=20)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        plt.show()


    def draw_degree_distribution_for_time_slot(self):
        degree_distribution = np.array([self.graph.degree(n) for n in self.graph.nodes()])
        sns.displot(degree_distribution, kde=True)
        plt.title('degree distribution')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.show()

    
    
    def rewire_graph(self):
        time_intervals = int(self.rewiring_number/self.time_slot_number)
        time_slots = list(range(0, self.rewiring_number+1, time_intervals))

        for iteration_number in range(self.rewiring_number):
            edge_to_rewire = random.choice(list(self.graph.edges()))
            node_i, node_j = edge_to_rewire

            node_i_connected_nodes = [edge[1] for edge in self.graph.edges(node_i)]
            node_i_connected_nodes.append(node_i)

            graph_nodes = list(self.graph.nodes)
            degrees = np.array([self.graph.degree(n) for n in graph_nodes])
            probability = self.fitness * (degrees + 1)

            for connected_node in node_i_connected_nodes:
                index = graph_nodes.index(connected_node)
                graph_nodes.pop(index)
                probability = np.delete(probability, index)

            probability_sum = np.cumsum(probability)
            random_number = np.random.uniform(0.0, probability_sum[len(probability_sum)-1])
            node_m_index = sum(i < random_number for i in probability_sum)

            node_m = graph_nodes[node_m_index]
            self.graph.remove_edge(*edge_to_rewire)
            self.graph.add_edge(node_i, node_m)
            
            if self.show_revolution and iteration_number in time_slots:
                print(f'iteration_number: {iteration_number}')
                self.draw_graph_for_time_slot()
                self.draw_degree_distribution_for_time_slot()
                time_slots = time_slots[1:]
        
    
    def __init__(self, node_number=100, edge_probability=0.5, fitness=np.array([]), rewiring_number=None, time_slot_number=10, seed=None, show_revolution=True):
        self.node_number = node_number
        self.edge_probability = edge_probability
        self.fitness = fitness
        self.time_slot_number = time_slot_number
        self.rewiring_number = rewiring_number if rewiring_number != None else np.power(node_number, 2)
        self.seed = seed if seed != None else random.randint(-100000000, 100000000)
        self.show_revolution = show_revolution
        self.graph = nx.erdos_renyi_graph(self.node_number, self.edge_probability, seed=self.seed)
        self.degree_per_slot = []
        self.rewire_graph()
        self.set_degree_distribution()
        self.draw_graph()

    def set_degree_distribution(self):
        self.degree_distribution = np.array([self.graph.degree(n) for n in self.graph.nodes()])

    def draw_degree_distribution(self, fitness_name=None):
        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        if fitness_name != None:
            plt.savefig(f'images/{fitness_name}-as-fitness-degree-distribution.jpg')
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution semilog')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.semilogy()
        plt.show()

        sns.displot(self.degree_distribution, kde=True)
        plt.title('degree distribution log-log')
        plt.xlabel('degree')
        plt.ylabel('occurance of each degree')
        plt.xscale('log')
        plt.yscale('log')
        plt.show()

    def draw_graph(self, fitness_name=None):
        fig = plt.figure(figsize=(50, 50))
        node_size = self.degree_distribution*100
        
        pos=nx.spring_layout(self.graph, k=0.15, iterations=20)
        cmap=plt.cm.viridis
        
        nodes = nx.draw_networkx_nodes(self.graph, pos, node_size=node_size, node_color=node_size, cmap=cmap)
        edges = nx.draw_networkx_edges(self.graph, pos)
        
        plt.colorbar(nodes)
        plt.axis('off')
        if fitness_name != None:
            plt.savefig(f'images/{fitness_name}-as-fitness-graph.jpg')
        plt.show()

## simulation

holding values to compare at the end

In [ ]:
graph_holder_dic = {}

### pareto as fitness

###### parameters

In [ ]:
a, m = 3., 2.  # shape and mode
node_number = 1000
edge_probability = 0.01
fitness = np.array([])
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating values

In [ ]:
fitness = (np.random.pareto(a, node_number) + 1) * m

wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['pareto'] = copy.deepcopy(wmg)

###### drawing degree distribution

In [ ]:
wmg.draw_degree_distribution('pareto')

###### drawing graph

In [ ]:
wmg.draw_graph('pareto')

### gamma as fitness

###### parameters

In [ ]:
shapea, scale = 2., 2.  # shape and mode
node_number = 1000
edge_probability = 0.01
fitness = np.array([])
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating values

In [ ]:
fitness = np.random.gamma(shape=shapea, scale=scale, size=node_number)

wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['gamma'] = copy.deepcopy(wmg)

###### drawing degree distribution

In [ ]:
wmg.draw_degree_distribution('gamma')

###### draw graph

In [ ]:
wmg.draw_graph('gamma')

### normal as fitness

###### parameters

In [ ]:
mu, sigma = 10, 0.1 # mean and standard deviation
node_number = 1000
edge_probability = 0.01
fitness = np.array([])
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating values

In [ ]:
fitness = np.random.normal(mu, sigma, node_number)

wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['normal'] = copy.deepcopy(wmg)

###### drawing degree distribution

In [ ]:
wmg.draw_degree_distribution('normal')

###### drawing graph

In [ ]:
wmg.draw_graph('normal')

### log-normal

###### parameters

In [ ]:
mu, sigma = 3., 1. # mean and standard deviation
node_number = 1000
edge_probability = 0.01
fitness = np.array([])
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating values

In [ ]:
fitness = np.random.lognormal(mu, sigma, node_number)

wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['log-normal'] = copy.deepcopy(wmg)

###### drawing degree distribution

In [ ]:
wmg.draw_degree_distribution('log-normal')

###### drawing graph

In [ ]:
wmg.draw_graph('log-normal')

### paretian-poisson burst process

##### generating paretian-poisson burst process

###### parameters

In [ ]:
total_time = 333
lam_list=[5.0]
hurst_list=[0.75]
burst_duration_mean = 4.0
has_pre_burst = True

###### generating

In [ ]:
ppbp_sim = PoissonParetoBurstProcessSimulator(total_time=total_time, lam_list=lam_list, hurst_list=hurst_list, burst_duration_mean=burst_duration_mean, has_pre_burst=has_pre_burst, least_interval_length=None)
sim_res = ppbp_sim.simulate()

##### generating rewiring model graph

###### parameters

In [ ]:
node_number = total_time * 3
edge_probability = 0.01
fitness = sim_res[0][0]
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating values

In [ ]:
wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['paretian-poisson burst process'] = copy.deepcopy(wmg)

###### drawing degree distribution

In [ ]:
wmg.draw_degree_distribution('paretian-poisson-burst-process')

###### drawing graph

In [ ]:
wmg.draw_graph('paretian-poisson-burst-process')

### paretian-poisson unified model

##### generating paretian-poisson unified model

###### parameter

In [ ]:
def power_law_trajectory_explosion_time(initial_value, specific_initial_value, epsiolon): # F(u)
    return initial_value / np.power(np.power(specific_initial_value/initial_value, -epsilon)-1, 1.0/epsilon)

largest_initial_val = 0.001 # v in paper
size = 1000
epsilon = 1.0
c = 1
start_interval = 0.99

###### generating paretian poisson

In [ ]:
initial_value_list = np.random.uniform(start_interval, largest_initial_val, size) * largest_initial_val

result = [power_law_trajectory_explosion_time(largest_initial_val, initial_value, epsilon) for initial_value in initial_value_list]

##### generating rewire model graph

###### parameters

In [ ]:
node_number = size
edge_probability = 0.01
fitness = np.array(result)
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating values

In [ ]:
wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['paretian-poisson unified-model'] = copy.deepcopy(wmg)

###### drawing degree distribution

In [ ]:
wmg.draw_degree_distribution('paretian-poisson-unified-model')

###### drawing graph

In [ ]:
wmg.draw_graph('paretian-poisson-unified-model')

### paretian-poisson pareto-poisson

##### generating pareto-poisson our model

###### parameters

In [ ]:
def equation3(U, v, epsilon):
    return v / np.power(np.power(U, -epsilon) - 1.0 , 1.0/epsilon)

def equation5(data, c, epsilon):
    return c * epsilon / np.power(data, 1.0 + epsilon)

def generate_our_pareto_poisson(v, epsilon, c, size, should_sort=False):
    U = np.random.uniform(0.0, 1.0, size-1) * v
    U = np.append(U, v)

    if should_sort:
        U = np.sort(U)
    
    X = equation3(U, v, epsilon)
    return equation5(X, c, epsilon)

v = .9# v in the paper
size = 1000
epsilon = 1.
c = 1.

final_min_value = 1
final_max_value = 40

###### generating pareto-poisson

In [ ]:
result = generate_our_pareto_poisson(v, epsilon, c, size, should_sort=True)

##### generating rewire model graph

###### parameters

In [ ]:
node_number = size
edge_probability = 0.01
fitness = np.array(result)
rewiring_number = 5000
time_slot_number = 10
show_revolution = True
seed = random.randint(-10000000, 10000000)

###### generating

In [ ]:
wmg = WiringModelGraph(node_number=node_number, edge_probability=edge_probability, fitness=fitness, rewiring_number=rewiring_number, time_slot_number=time_slot_number, seed=seed, show_revolution=show_revolution)

graph_holder_dic['paretian-poisson our model'] = copy.deepcopy(wmg)

###### degree distribution

In [ ]:
wmg.draw_degree_distribution()

###### graph

In [ ]:
wmg.draw_graph()

### comparison

###### simple plot

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8))
    step_dict = {}
    for fitness_function, graph_holder in graph_holder_dic.items():
        step_dict[fitness_function] = graph_holder.degree_per_slot[step_number]

    expected_len = 0

    for fitness_function, graph_holder in step_dict.items():
        expected_len = max(expected_len, len(graph_holder))

    for fitness_function, graph_holder in step_dict.items():
        if len(graph_holder) < expected_len:
            step_dict[fitness_function] = np.append(graph_holder, np.NaN)

    step_df = pd.DataFrame.from_dict(step_dict)
    sns.displot(data = step_df, stat="probability",height=6,aspect=2.67)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.xlabel("Degree")
    plt.ylabel("# of Nodes")
    plt.legend(loc='best')
    plt.show()

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8))
    step_dict = {}
    for fitness_function, graph_holder in graph_holder_dic.items():
        step_dict[fitness_function] = graph_holder.degree_per_slot[step_number]

    expected_len = 0

    for fitness_function, graph_holder in step_dict.items():
        expected_len = max(expected_len, len(graph_holder))

    for fitness_function, graph_holder in step_dict.items():
        if len(graph_holder) < expected_len:
            step_dict[fitness_function] = np.append(graph_holder, np.NaN)

    step_df = pd.DataFrame(step_dict)
    sns.displot(data = step_df, kind="kde",height=6,aspect=2.67)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.xlabel("Degree")
    plt.ylabel("# of Nodes")
    plt.legend(loc='best')
    plt.show()

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8))
    step_dict = {}
    for fitness_function, graph_holder in graph_holder_dic.items():
        step_dict[fitness_function] = graph_holder.degree_per_slot[step_number]

    expected_len = 0

    for fitness_function, graph_holder in step_dict.items():
        expected_len = max(expected_len, len(graph_holder))

    for fitness_function, graph_holder in step_dict.items():
        if len(graph_holder) < expected_len:
            step_dict[fitness_function] = np.append(graph_holder, np.NaN)

    step_df = pd.DataFrame(step_dict)
    sns.displot(data = step_df, kind="kde", fill=True,height=6,aspect=2.67)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.xlabel("Degree")
    plt.ylabel("# of Nodes")
    plt.legend(loc='best')
    plt.show()

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8))
    step_dict = {}
    for fitness_function, graph_holder in graph_holder_dic.items():
        step_dict[fitness_function] = graph_holder.degree_per_slot[step_number]

    expected_len = 0

    for fitness_function, graph_holder in step_dict.items():
        expected_len = max(expected_len, len(graph_holder))

    for fitness_function, graph_holder in step_dict.items():
        if len(graph_holder) < expected_len:
            step_dict[fitness_function] = np.append(graph_holder, np.NaN)

    step_df = pd.DataFrame(step_dict)
    sns.displot(data = step_df, kind="kde", multiple="stack",height=6,aspect=2.67)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.xlabel("Degree")
    plt.ylabel("# of Nodes")
    plt.legend(loc='best')
    plt.show()

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8))
    for fitness_function, graph_holder in graph_holder_dic.items():
        plt.plot(np.sort(graph_holder.degree_per_slot[step_number])[::-1], label=fitness_function)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.ylabel("Degree")
    plt.legend(loc='best')
    plt.show()

###### semilog plot

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8))
    for fitness_function, graph_holder in graph_holder_dic.items():
        plt.plot(np.sort(graph_holder.degree_per_slot[step_number])[::-1], label=fitness_function)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.ylabel("Degree")
    plt.semilogy()
    plt.legend(loc='best')
    plt.show()

###### log-log plot

In [ ]:
for step_number in range(time_slot_number):
    plt.figure(figsize=(12, 8)) 
    for fitness_function, graph_holder in graph_holder_dic.items():
        plt.loglog(np.sort(graph_holder.degree_per_slot[step_number])[::-1], label=fitness_function)
    plt.title(f'degree distribution comparison for step: {step_number}')
    plt.ylabel("Degree")
    plt.legend(loc='best')
    plt.show()